In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

In [2]:
base_dir = 'D:\\sk sir\\lab2\\human_action'

In [3]:
# Initialize the ImageDataGenerator with a validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Use 20% for testing/validation

# Load training data (80%)
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # Resize images
    batch_size=32,
    class_mode='categorical',  # Use 'binary' for binary classification
    subset='training'  # Load the training data (80% of the images)
)

# Load validation data (20%)
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),  # Resize images
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Load the validation data (20% of the images)
)

Found 5148 images belonging to 9 classes.
Found 1278 images belonging to 9 classes.


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the pre-trained MobileNetV2 model without the top classification layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model's layers to prevent them from being updated during training
base_model.trainable = False

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the full model by combining the base model and the new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with an optimizer, loss function, and metrics for evaluation
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data, using the test data for evaluation
model.fit(train_generator, validation_data=validation_generator, epochs=20)

Epoch 1/20


C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 154s 915ms/step - accuracy: 0.6222 - loss: 1.1560 - val_accuracy: 0.7066 - val_loss: 0.7725
Epoch 2/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.8122 - loss: 0.5284 - val_accuracy: 0.7856 - val_loss: 0.6531
Epoch 3/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 67s 416ms/step - accuracy: 0.8850 - loss: 0.3538 - val_accuracy: 0.7856 - val_loss: 0.6167
Epoch 4/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 68s 424ms/step - accuracy: 0.9098 - loss: 0.2699 - val_accuracy: 0.7973 - val_loss: 0.6323
Epoch 5/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 98s 607ms/step - accuracy: 0.9406 - loss: 0.1861 - val_accuracy: 0.7778 - val_loss: 0.7329
Epoch 6/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.9641 - loss: 0.1299 - val_accuracy: 0.7833 - val_loss: 0.7466
Epoch 7/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.9860 - loss: 0.0726 - val_accuracy: 0.8059 - val_loss: 0.7092
Epoch 8/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 353s 2s/step - accuracy: 0.9877 - loss: 0.0505 - val_accura

In [5]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(validation_generator)
print(f'Test Accuracy: {accuracy:.2f}')

40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 320ms/step - accuracy: 0.8460 - loss: 0.7736
Test Accuracy: 0.82


In [7]:
model.save('model.h5')